In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import collections
import socket
import struct
import os
import pickle
import time

In [7]:
ScalerPath = "./Model/Scaler.pickle"
ModelPath = "./Model/Clf_Model.pickle"
HeaderWithoutAccel = ["ApproachRateOther", 
                      "Rel_Pos_Magnitude", 
                      "1_Head_Center_Distance", "2_Head_Center_Distance", 
                      # "Filtered_1_Head_Velocity_Total",
                      "Filtered_2_Head_Velocity_Total"
                    #   "1_Turn",
                    #   "2_Indicator",
                    #   "Centerline_Offset_2",
                    #   "RelativeRotation"
                      ] # Signaling ()

# outputHeader = ["Centerline_Offset_A"]
outputHeader = ["Cat_Accel1"]
input_size = len(HeaderWithoutAccel)
hidden_size = 6
TargetTimeLength = 0.75 ## in seconds
Seq_Length = 13
output_size = len(outputHeader)

In [5]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU layer: processes sequences
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize the hidden state for the first sequence
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate through GRU
        out, _ = self.rnn(x, h0)  # out shape: (batch_size, sequence_length, hidden_size)

        # Use the hidden state from the last time step
        out = out[:, -1, :]  # Shape: (batch_size, hidden_size)

        # Pass through the fully connected layer
        out = self.fc(out)  # Shape: (batch_size, output_size)
        
        prob = F.softmax(out, dim=1)
        return prob


In [6]:
scaler=None
if os.path.exists(ScalerPath):
    with open(ScalerPath, 'rb') as file:
        scaler = pickle.load(file)
else:
    print ("Could NOT find Scaler")

In [8]:
# Set parameters
input_size = 5  # Number of features per time step
hidden_size = 32  # Number of GRU units
output_size = 2   # Number of output classes for binary classification
num_layers = 1   # Number of GRU layers
num_epochs = 10
batch_size = 64
seq_length = 13   # Length of each input sequence

# Create the model
net = GRUModel(input_size, hidden_size, output_size, num_layers)
net.load_state_dict(torch.load(ModelPath))
net.eval()
d = collections.deque(maxlen=Seq_Length)

/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_18409/3919113624.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ModelPath))


In [9]:
# Initial Status
for i in range(10):
    d.append(np.ones(input_size)+i)
scaled_features = scaler.transform(np.array(d))
net(torch.Tensor([scaled_features]))

/var/folders/py/bm4kf8_j7tn54t9vvg2pyb5r0000gn/T/ipykernel_18409/753876682.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1720538210021/work/torch/csrc/utils/tensor_new.cpp:281.)
  net(torch.Tensor([scaled_features]))


tensor([[3.3310e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [10]:
UDP_IP = "127.0.0.1"
txPort = 8000
rxPort = 8001
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, rxPort))

In [11]:
while True:
    try:
        data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
       
        if(int(len(data)/4)!=7):
            print("Didnt get enoughb bytes, continuing!",int(len(data)/4))
            continue
        array = struct.unpack('fffffff', data) 

        d.append(np.array(array))
        scaled_features = scaler.transform(np.array(d))
        val = net(torch.Tensor([scaled_features]))
        floatlist=[val]
        #print(array,val)
        buf = struct.pack('%sf' % len(floatlist), *floatlist)
        sock.sendto(buf, (UDP_IP, txPort))
    except BlockingIOError:
        time.sleep(1/19)
        

KeyboardInterrupt: 